In [1]:
import numpy as np
import pandas as pd
import math
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multinomial
from typing import List
import seaborn as sns

## system characterisation ##

class State: 
    def __init__(self,mean,eta,lambda_switch,sigma):
        self.m = mean
        self.e = eta 
        self.l = lambda_switch
        self.s = sigma

M = 2
N = 2

state_1 = State(3,0.1,0.1,0.1)

def Profit(E,P,vc,fc,pi_tax,c_tax):
    pi = ((E*(P-vc-c_tax))-fc)*(1-pi_tax)
    return pi

T = 30
E = 2500
r = 0.05 
vc = 1
fc = 20
pi_tax = 0.2
c_tax = 1

switch = (0.2,0.6)

## node construction ##

class node:
    def __init__(self,value,temp):
        self.value = v
        self.temp = t

## finite difference parameter definition ##

## parameter defn for P_i in i = 2,...,i_max-1 ##

def difference(P,p_step,sigma,eta,mean):
    difference = np.zeros(2)
    a_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    b_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    if a_i_central > 0:
        difference[0] = a_i_central
    else:
        difference[0] = a_i_forward
    if b_i_central > 0:
        difference[1] = b_i_central
    else:
        difference[1] = b_i_backward
    return(difference)

## parameter defn for P_i in i = 1 ##
    
def difference_min(P,p_step,eta,mean):
    a_i = 0
    b_i = eta*(mean-P)/(p_step)
    return [a_i,b_i]

## parameter defn for P_i in i = i_max ##

def difference_max(P,p_step,eta,mean):
    a_i = abs(eta*(mean - P)/(p_step))
    b_i = 0
    return [a_i,b_i]

## lattice parameters ##

p_step = 0.1
s0_step = 100000

p_1 = 0.5
p_max = 8
p_steps = int((p_max-p_1)/p_step)

s0_1 = 100000
s0_max = 500000

## lattice construction ##

P_grid = np.linspace(p_1,p_max,p_steps)
s0_grid = list(range(s0_1,s0_max,s0_step))

temp_node = np.full((len(P_grid),len(s0_grid),M,N),0)
node_value = np.full((len(P_grid),len(s0_grid),M,N),0)
Lv = np.full((len(P_grid),len(s0_grid),M,N),0)

In [2]:
for q in range(4):
        
        ## Quarterly profit at each node ## 
        
        for i in range(len(P_grid)):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        if n == 1:
                            temp_node[i,k,m,n] = temp_node[i,k,m,n] + Profit(E,P_grid[i],vc,fc,pi_tax,c_tax)
                        else:
                            temp_node[i,k,m,n] = temp_node[i,k,m,n] - fc
        
## Lagrange Differential at each node ##
        
        
## we loop over Lagrange differential for boundary nodes ##                

        for k in range(len(s0_grid)):
            for m in range(M):
                for n in range(N):
                    diff = difference_min(P_grid[0],p_step,state_1.e,state_1.m)
                    print(diff)
                    Lv[0,k,m,n] = diff[1]*temp_node[1,k,m,n] - (diff[1]+r)*temp_node[0,k,m,n]

        for k in range(len(s0_grid)):
            for m in range(M):
                for n in range(N):
                    diff = difference_max(len(P_grid) - 1,p_step,state_1.e,state_1.m)
                    print(diff)
                    Lv[len(P_grid) - 1,k,m,n] = diff[0]*temp_node[len(P_grid) - 2,k,m,n]  - (diff[0]+r)*temp_node[len(P_grid) - 1,k,m,n]

## then we loop over lagrange differential for nodes inside the boundary ##
    
        for i in range(1,len(P_grid) - 1):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        diff = difference(P_grid[i],p_step,state_1.s,state_1.e,state_1.m)
                        print(diff)
                        Lv[i,k,m,n] = diff[0]*temp_node[i - 1,k,m,n] + diff[1]*temp_node[i + 1,k,m,n] - (diff[0]+diff[1]+r)*temp_node[i,k,m,n] 
                    

## then we apply Lv to temporary node values ##
        for i in range(len(P_grid)):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                         temp_node[i,k,m,n] = temp_node[i,k,m,n] + Lv[i,k,m,n]  

[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[0, 2.5]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[71.0, 0]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.18081172 0.18081172]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]
[0.24689554 0.24689554]


[28.72126096 28.72126096]
[28.72126096 28.72126096]
[28.72126096 28.72126096]
[28.72126096 28.72126096]
[28.72126096 28.72126096]
[28.72126096 28.72126096]
[28.72126096 28.72126096]
[28.72126096 28.72126096]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[29.51597882 29.51597882]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877 30.32096877]
[30.32096877

[14.98404401 14.98404401]
[14.98404401 14.98404401]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[15.57331994 15.57331994]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.17286797 16.17286797]
[16.78268809 16.78268809]
[16.78268809 16.78268809]
[16.78268809 16.78268809]
[16.78268809 16.78268809]
[16.78268809

[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[6.95263422 6.95263422]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.37755661 7.37755661]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511]
[7.8127511 7.8127511

OverflowError: Python int too large to convert to C long

In [3]:
print(temp_node)

[[[[       -76      -9827]
   [       -76      -9827]]

  [[       -76      -9827]
   [       -76      -9827]]

  [[       -76      -9827]
   [       -76      -9827]]

  [[       -76      -9827]
   [       -76      -9827]]]


 [[[       -76     -10346]
   [       -76     -10346]]

  [[       -76     -10346]
   [       -76     -10346]]

  [[       -76     -10346]
   [       -76     -10346]]

  [[       -76     -10346]
   [       -76     -10346]]]


 [[[       -76      -9661]
   [       -76      -9661]]

  [[       -76      -9661]
   [       -76      -9661]]

  [[       -76      -9661]
   [       -76      -9661]]

  [[       -76      -9661]
   [       -76      -9661]]]


 ...


 [[[     -6343  -16021871]
   [     -6343  -16021871]]

  [[     -6343  -16021871]
   [     -6343  -16021871]]

  [[     -6343  -16021871]
   [     -6343  -16021871]]

  [[     -6343  -16021871]
   [     -6343  -16021871]]]


 [[[      4234   65264667]
   [      4234   65264667]]

  [[      4234   65264667]
   [  